# Data modelling on Bike Sharing data in London

Data aquired from: https://s3.amazonaws.com/capitalbikeshare-data/index.html  
Data project initiated: 25/01/2019  
Author: Sedar Olmez

Data modelling:  
    GDS1: Data Gathering, Preparation and Exploration.  
    GDS2: Data Representation and Transformation.  
    GDS3: Computing with Data.   
    GDS4: Data Visualisation and Presentation.   
    GDS5: Data Modelling.   
    GDS6: Science about Data Science. 

Assessment:
![Assessment](assessment.png)

# Dataset information
I have two datasets, one I will be using to clean and the other to perform machine learning algorithms on. The reason for why I am using two is because the original dataset I want to use has already been cleaned, therefore, I need to perform some cleaning tasks on a smaller dataset with a similar theme.

1) Dataset (cleaning): Walking_Cycling.csv - London Cycling % and Walking % by Local Authority `COLUMNS: 'LA code', 'Local Authority', 'Year', 'Frequency', 'Walking_%', 'Cycling_%'`


2) Dataset (Machine Learning/visualisation): capitalbikeshare-tripdata-washington.csv - Capital Bike sharing information in Washington DC 2017, large dataset with a lot of useful data.

In [67]:
# Libraries
from __future__ import print_function
import matplotlib.pyplot as plt
import seaborn as sea
import pandas as pd
import numpy as np
from datetime import date
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor as DTR

## Small dataset - Walking_Cycling.csv

In [48]:
%%time
# Load up the journeys.csv dataset into dataframes
dataframe_journeys = pd.read_csv('data/Walking-Cycling.csv')
dataframe_journeys.columns = ['LA code', 'Local Authority', 'Year', 'Frequency', 'Walking_%', 'Cycling_%']
dataframe_journeys.dropna()

CPU times: user 7.02 ms, sys: 2.5 ms, total: 9.52 ms
Wall time: 7.79 ms


In [49]:
dataframe_journeys.head()

,LA code,Local Authority,Year,Frequency,Walking_%,Cycling_%
0,E09000001,City of London,2010/11,1x per month,78,30
1,E09000002,Barking and Dagenham,2010/11,1x per month,60,8
2,E09000003,Barnet,2010/11,1x per month,65,10
3,E09000004,Bexley,2010/11,1x per month,65,11
4,E09000005,Brent,2010/11,1x per month,62,14


In [50]:
list(dataframe_journeys)

['LA code', 'Local Authority', 'Year', 'Frequency', 'Walking_%', 'Cycling_%']

In [51]:
# We will now convert the Local Authority column to string
dataframe_journeys['Local Authority'] = dataframe_journeys['Local Authority'].astype('|S')

In [52]:
dataframe_journeys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204 entries, 0 to 1203
Data columns (total 6 columns):
LA code            1204 non-null object
Local Authority    1204 non-null object
Year               1204 non-null object
Frequency          1204 non-null object
Walking_%          1204 non-null int64
Cycling_%          1204 non-null object
dtypes: int64(1), object(5)
memory usage: 56.5+ KB


#### In Pandas, dtype(obj) == python.dtype(str) therefore we set the string to the maximum bytes of the longest string stored by using

```.astype(|S)```

In [56]:
# We must now convert both the Walking_% and Cycling_% to float as this would make regression more accurate.
# However we must first convert Cycling_% from String to Int then Float.
dataframe_journeys['Cycling_%'].astype(int)

ValueError: invalid literal for long() with base 10: '-'

In [57]:
# An invalit literal for long() error is thrown, this means there are values in the column which cannot be converted to int
# Let us identify these columns
dataframe_journeys['Cycling_%'] = pd.to_numeric(dataframe_journeys['Cycling_%'], errors='coerce')

In [59]:
# We found the row with the problem, for row 326 the column Cycling_% had '-' so we replaced it with NaN.
print (dataframe_journeys[ pd.to_numeric(dataframe_journeys['Cycling_%'], errors='coerce').isnull()])

       LA code Local Authority     Year    Frequency  Walking_%  Cycling_%
326  E09000026       Redbridge  2011/12  5x per week         17        NaN


In [74]:
# We must replace the NaN to an integer value i.e. 0 so we can produce a pairplot later using seaborn.
dataframe_journeys = dataframe_journeys.fillna(0.0)

In [75]:
# The 'Cycling_%' column was converted to int, now we can focus on changing both columns to floats.
dataframe_journeys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204 entries, 0 to 1203
Data columns (total 6 columns):
LA code            1204 non-null object
Local Authority    1204 non-null object
Year               1204 non-null object
Frequency          1204 non-null object
Walking_%          1204 non-null float64
Cycling_%          1204 non-null float64
dtypes: float64(2), object(4)
memory usage: 56.5+ KB


In [76]:
# Here we convert Walking_% to float from int
dataframe_journeys['Walking_%'] = dataframe_journeys['Walking_%'].astype(float)

In [77]:
# As can be seen, we converted the columns to floats.
dataframe_journeys.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204 entries, 0 to 1203
Data columns (total 6 columns):
LA code            1204 non-null object
Local Authority    1204 non-null object
Year               1204 non-null object
Frequency          1204 non-null object
Walking_%          1204 non-null float64
Cycling_%          1204 non-null float64
dtypes: float64(2), object(4)
memory usage: 56.5+ KB


In [78]:
dataframe_journeys.head()

,LA code,Local Authority,Year,Frequency,Walking_%,Cycling_%
0,E09000001,City of London,2010/11,1x per month,78.0,30.0
1,E09000002,Barking and Dagenham,2010/11,1x per month,60.0,8.0
2,E09000003,Barnet,2010/11,1x per month,65.0,10.0
3,E09000004,Bexley,2010/11,1x per month,65.0,11.0
4,E09000005,Brent,2010/11,1x per month,62.0,14.0


## Dataset cleaned and optimised, now to explore we will use our main dataset on the bikeshare data in washington DC for the year 2017 (already cleaned)

In [112]:
%%time
# Load up the journeys.csv dataset into dataframes
dataframe_washington = pd.read_csv('data/2017-capitalbikeshare-tripdata-washington.csv')
dataframe_washington.columns = ['Duration', 'Start Date', 'End Date', 'Start Station Number', 'Start Station Name', 'End Station Number',
                                'End Station Name', 'Bike Number', 'Member Type']
dataframe_washington.dropna()

CPU times: user 2.81 s, sys: 138 ms, total: 2.95 s
Wall time: 1.39 s


In [113]:
dataframe_washington.head()

,Duration,Start Date,End Date,Start Station Number,Start Station Name,End Station Number,End Station Name,Bike Number,Member Type
0,221,2017-01-01 00:00:41,2017-01-01 00:04:23,31634,3rd & Tingey St SE,31208,M St & New Jersey Ave SE,W00869,Member
1,1676,2017-01-01 00:06:53,2017-01-01 00:34:49,31258,Lincoln Memorial,31270,8th & D St NW,W00894,Casual
2,1356,2017-01-01 00:07:10,2017-01-01 00:29:47,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W21945,Casual
3,1327,2017-01-01 00:07:22,2017-01-01 00:29:30,31289,Henry Bacon Dr & Lincoln Memorial Circle NW,31222,New York Ave & 15th St NW,W20012,Casual
4,1636,2017-01-01 00:07:36,2017-01-01 00:34:52,31258,Lincoln Memorial,31270,8th & D St NW,W22786,Casual


In [114]:
dataframe_washington.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646510 entries, 0 to 646509
Data columns (total 9 columns):
Duration                646510 non-null int64
Start Date              646510 non-null object
End Date                646510 non-null object
Start Station Number    646510 non-null int64
Start Station Name      646510 non-null object
End Station Number      646510 non-null int64
End Station Name        646510 non-null object
Bike Number             646510 non-null object
Member Type             646510 non-null object
dtypes: int64(3), object(6)
memory usage: 44.4+ MB


In [115]:
# We can strip the 'W' from the Bike Numbers to convert it into an Integer which we can then add to our visualisation 
# Data manipulation for later.
dataframe_journeys['Bike Number'] = dataframe_washington['Bike Number'].str[1:]

In [120]:
# I now converted the Bike Number column to integer so we can add it to our data analysis.
dataframe_journeys['Bike Number'] = pd.to_numeric(dataframe_journeys['Bike Number'], downcast='integer')